<a href="https://colab.research.google.com/github/tolgakurtuluss/LangChain_chatwithcsv_irisdataset/blob/main/LangChain_with_iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai
!pip show langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.2/477.2 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 4.8 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-PLACEYOUROWNAPI"

In [3]:
import pandas as pd

df = pd.read_csv('/content/iris.csv')

In [4]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [5]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [6]:
agent = create_csv_agent(OpenAI(temperature=0), 
                         '/content/iris.csv', 
                         verbose=True)

In [7]:
agent

AgentExecutor(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f94e0cc9610>, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f94e0cc9610>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df': '   sepal.length  sepal.width  petal.length  petal.width variety\n0           5.1          3.5           1.4          0.2  Setosa\n1           4.9          3.0           1.4          0.2  Setosa\n2           4.7          3.2           1.3          0.2  Setosa\n3           4.6          3.1           1.5          0.2  Setosa\n4           5.0          3.6           1.4          0.2  Setosa'}, template='\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python s

In [8]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [9]:
agent.run("Explain this dataset statistically with mentioning outliers, mean and unique features of values.")



> Entering new AgentExecutor chain...
Thought: I should look at the data to get an understanding of the values.
Action: python_repl_ast
Action Input: df.describe()
Observation:        sepal.length  sepal.width  petal.length  petal.width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000
Thought: I now have an understanding of the data.
Final Answer: This dataset contains 150 observations of 4 features (sepal.length, sepal.width, petal.length, petal.width). The mean values for each feature are 5.84 for sepal.length, 3.06 for sepal.width, 3.76 for petal.length, and

'This dataset contains 150 observations of 4 features (sepal.length, sepal.width, petal.length, petal.width). The mean values for each feature are 5.84 for sepal.length, 3.06 for sepal.width, 3.76 for petal.length, and 1.20 for petal.width. The minimum and maximum values for each feature are 4.30 and 7.90 for sepal.length, 2.00 and 4.40 for sepal.width, 1.00 and 6.90 for petal.length, and 0.10 and 2.50 for petal.width, respectively. There may be outliers in the dataset, as the maximum values for each feature are significantly higher than the 75th percentile values.'

In [10]:
agent.run("Do a detailed analysis for explaining the key points of 4 features in this dataset.")



> Entering new AgentExecutor chain...
Thought: I should look at the data and see what I can find.
Action: python_repl_ast
Action Input: df.describe()
Observation:        sepal.length  sepal.width  petal.length  petal.width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000
Thought: I can see the mean, min, max, and quartiles for each feature.
Action: python_repl_ast
Action Input: df.variety.value_counts()
Observation: Setosa        50
Versicolor    50
Virginica     50
Name: variety, dtype: int64
Thought: I can also see the number of each variety.
Final Answer: Th

'The sepal length has a mean of 5.84, a min of 4.3, and a max of 7.9. The sepal width has a mean of 3.06, a min of 2, and a max of 4.4. The petal length has a mean of 3.76, a min of 1, and a max of 6.9. The petal width has a mean of 1.2, a min of 0.1, and a max of 2.5. There are 50 of each variety in the dataset.'

In [11]:
agent.run("Do a correlation test for 4 features and explain their significance.")



> Entering new AgentExecutor chain...
Thought: I need to calculate the correlation between the 4 features.
Action: python_repl_ast
Action Input: df.corr()
Observation:               sepal.length  sepal.width  petal.length  petal.width
sepal.length      1.000000    -0.117570      0.871754     0.817941
sepal.width      -0.117570     1.000000     -0.428440    -0.366126
petal.length      0.871754    -0.428440      1.000000     0.962865
petal.width       0.817941    -0.366126      0.962865     1.000000
Thought: I can interpret the correlation values.
Final Answer: The correlation test shows that sepal length and petal length have a strong positive correlation (0.871754), while sepal width and petal width have a moderate negative correlation (-0.366126). This suggests that sepal length and petal length are related, while sepal width and petal width are inversely related.

> Finished chain.


'The correlation test shows that sepal length and petal length have a strong positive correlation (0.871754), while sepal width and petal width have a moderate negative correlation (-0.366126). This suggests that sepal length and petal length are related, while sepal width and petal width are inversely related.'